# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv", encoding = 'utf-8')

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,flying fish cove,-10.4217,105.6791,301.97,74,75,3.60,CX,1705115821
1,1,west island,-12.1568,96.8225,301.14,94,40,2.57,CC,1705115821
2,2,hermanus,-34.4187,19.2345,291.80,89,0,0.58,ZA,1705115822
3,3,santa cruz del norte,23.1539,-81.9236,296.86,91,13,4.59,CU,1705115822
4,4,gbarnga,6.9956,-9.4722,295.03,100,34,0.90,LR,1705115822


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    color="City",
    size="Humidity",
    scale=1,
    alpha=0.5,
    tiles='OSM',
    hover_cols=["City"],
    geo= True,
    title="City Map with Humidity",
    xlabel="Longitude",
    ylabel="Latitude",

)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Ideal weather conditions 
ideal_conditions = {
    "Temperature": (100, 300),  
    "Wind Speed": (0, 10),    
    "Cloudiness": 0            
}

# DataFrame based on the ideal conditions
ideal_cities_df = city_data_df[
    (city_data_df["Max Temp"].between(*ideal_conditions["Temperature"])) &
    (city_data_df["Wind Speed"] < ideal_conditions["Wind Speed"][1]) &
    (city_data_df["Cloudiness"] == ideal_conditions["Cloudiness"])
]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,hermanus,-34.4187,19.2345,291.80,89,0,0.58,ZA,1705115822
20,20,cruzeiro do sul,-29.5128,-51.9853,292.86,85,0,1.21,BR,1705115843
25,25,hawaiian paradise park,19.5933,-154.9731,296.98,67,0,9.26,US,1705115845
35,35,bredasdorp,-34.5322,20.0403,291.69,92,0,0.70,ZA,1705115847
54,54,badger,64.8000,-147.5333,255.27,75,0,0.00,US,1705115852


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
2,2,hermanus,-34.4187,19.2345,291.80,89,0,0.58,ZA,1705115822,
20,20,cruzeiro do sul,-29.5128,-51.9853,292.86,85,0,1.21,BR,1705115843,
25,25,hawaiian paradise park,19.5933,-154.9731,296.98,67,0,9.26,US,1705115845,
35,35,bredasdorp,-34.5322,20.0403,291.69,92,0,0.70,ZA,1705115847,
54,54,badger,64.8000,-147.5333,255.27,75,0,0.00,US,1705115852,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000  # You can adjust the radius as needed
params = {
    "categories": "lodging",
    "radius": radius,
    "limit": 1,
    "apiKey": "9b0702abf0b64fc0aa09a3248fcf3306E", 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"point=({longitude},{latitude})"
    params["bias"] = f"point=({longitude},{latitude})"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params).json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()


Starting hotel search
hermanus - nearest hotel: No hotel found
cruzeiro do sul - nearest hotel: No hotel found
hawaiian paradise park - nearest hotel: No hotel found
bredasdorp - nearest hotel: No hotel found
badger - nearest hotel: No hotel found
tamanrasset - nearest hotel: No hotel found
tura - nearest hotel: No hotel found
crucecita - nearest hotel: No hotel found
dali - nearest hotel: No hotel found
baihar - nearest hotel: No hotel found
ubari - nearest hotel: No hotel found
mantua - nearest hotel: No hotel found
yueyang - nearest hotel: No hotel found
harindanga - nearest hotel: No hotel found
dwarka - nearest hotel: No hotel found
sylvan lake - nearest hotel: No hotel found
pedernales - nearest hotel: No hotel found
porbandar - nearest hotel: No hotel found
san jose de feliciano - nearest hotel: No hotel found
castillos - nearest hotel: No hotel found
vallenar - nearest hotel: No hotel found
zaragoza - nearest hotel: No hotel found
pizhou - nearest hotel: No hotel found
altai - 

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
2,2,hermanus,-34.4187,19.2345,291.80,89,0,0.58,ZA,1705115822,No hotel found
20,20,cruzeiro do sul,-29.5128,-51.9853,292.86,85,0,1.21,BR,1705115843,No hotel found
25,25,hawaiian paradise park,19.5933,-154.9731,296.98,67,0,9.26,US,1705115845,No hotel found
35,35,bredasdorp,-34.5322,20.0403,291.69,92,0,0.70,ZA,1705115847,No hotel found
54,54,badger,64.8000,-147.5333,255.27,75,0,0.00,US,1705115852,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display
# Configure the map plot with hotel name and country in the hover tooltip
map_plot_with_hotels = hotel_df.hvplot.scatter(
    x="Lng",
    y="Lat",
    c="Humidity",
    size="Humidity",
    hover_cols=["City", "Country", "Hotel Name"],
    title="City Map with Humidity and Hotel Information",
    xlabel="Longitude",
    ylabel="Latitude",
    clim=(0, 100),
    cmap="viridis",
)

# Display the map with hotel information in the hover tooltip
map_plot_with_hotels


:Scatter   [Lng]   (Lat,Humidity,City,Country,Hotel Name)